In [5]:
import casadi 
import pandas as pd
import numpy as np

In [6]:
def Sistemas(x0, P1_value, lut):
    x = MX.sym('x', 2)  
    p = MX.sym('p', 2)

    # Interpolação
    phi_value = lut([p[0], p[1]])  # Interpolação usando N_rot e Mass
    
    # Constantes
    A1 = 2.6e-3   # m²
    Lc = 2.0      # m
    alpha = 0.5   
    P_out = 5.0   
    C1 = 479.029  
    nu_P = 2.0    # m³
    K_nu = 0.38 * 1000  # kg / (kBar^0.5 s)

    # Definir `z`
    z = if_else(x[1] < P_out, 0, sqrt(x[1] - P_out))
    
    # Definir RHS da EDO
    rhs = vertcat(
        (A1 / Lc) * (vertcat(p[1], x[0]) * p[0] - (x[1])),
        (C1**2) * (x[0] - alpha * K_nu * z * phi_value)
    ) / nu_P

    # Criar o integrador
    ode = {'x': x, 'ode': rhs, 'p': p}
    F = integrator('F', 'cvodes', ode, {'tf': 90})

    # Avaliar
    t_eval = np.linspace(0, 90, 120)
    sol = np.zeros((len(t_eval), 2))
    zeronum = 1e-9

    for i in range(len(t_eval)):
        res = F(x0=x0, p=P1_value)
        x_next = res['xf'].full().flatten()
        sol[i] = x_next

        # Verificar estado estacionário
        if np.all(np.abs(x_next - x0) < zeronum):
            sol = sol[:i+1]
            t_eval = t_eval[:i+1]
            break
        x0 = x_next

    return sol, P1_value, t_eval


In [7]:
def Loop(loops, stati, lut):
    estados = []
    p1_total = []
    t_eval_total = []

    for i in range(loops):
        if i == 0:
            x0 = stati  # Estado inicial
        else:
            x0 = estados[-1][-1]

        P1_v = p1_gen.P1()  # Geração aleatória de P1
        sol, P1_aleatorio, t_eval = Sistemas(x0, P1_v, lut)
        estados.append(sol)
        p1_total.append(P1_aleatorio)
        
        # Concatenar tempo
        if i == 0:
            t_total = t_eval
        else:
            t_total = np.concatenate([t_total, t_total[-1] + t_eval])

    return estados, p1_total, t_total
